In [1]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl

In [2]:
import os
import torch
import pandas as pd
from datasets import load_dataset
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [3]:
base_model = "NousResearch/Llama-2-7b-chat-hf"

new_model = "llama-2-7b-chat-Analysys-VC"

In [4]:
data = [{"input": "Accel", "output": "You're interested in Accel, a venture capital firm. Here are some key facts about Accel: 1. Founding: Accel was founded in 1983 by Jim Breyer, Arthur Patterson, and Harry Weller. 2. Focus: Accel focuses on investing in and supporting early-stage technology startups. 3. Portfolio: Accel has invested in a wide range of startups, including companies like Facebook, Spotify, and Airbnb. 4. Funding: Accel has raised multiple funds, including a $2.5 billion fund in 2018 and a $4.4 billion fund in 2020. 5. Team: Accel has a team of experienced investors, entrepreneurs, and industry experts. The firm is known for its collaborative approach to investing and its ability to provide support and guidance to its portfolio companies."}
,{"input": "Andreessen Horowitz", "output": "A16z is a venture capital firm that focuses on investing in and supporting early-stage startups. Here are some key facts about A16z: 1. Founding: A16z was founded in 2009 by Marc Andreessen and Ben Horowitz. 2. Focus: A16z focuses on investing in and supporting early-stage startups, particularly in the areas of software, data science, and artificial intelligence. 3. Portfolio: A16z has invested in a wide range of startups, including companies like Facebook, Twitter, and Airbnb. 4. Funding: A16z has raised multiple funds, including a $1.5 billion fund in 2017 and a $2.7 billion fund in 2020. 5. Team: A16z has a team of experienced investors, entrepreneurs, and industry experts. The firm is known for its collaborative approach to investing and its ability to provide support and guidance to its portfolio companies."}
,{"input": "Greylock Partners", "output": "Greylock Partners is a prominent venture capital firm based in Silicon Valley. They are known for being early partners of remarkable teams building companies that have a significant impact. Greylock has a strong track record of investing in successful companies such as Facebook, Airbnb, Aurora, and many others. The firm focuses on investing in technology companies across various sectors, including enterprise software, consumer internet, healthcare, and more. They provide funding and support to help these companies grow and succeed. Greylock Partners has a diverse portfolio of investments and works closely with founders and management teams to help them navigate challenges and scale their businesses. They offer expertise, resources, and a network of connections to help their portfolio companies thrive. Additionally, Greylock Partners is actively involved in the startup community and regularly shares insights and advice through their blog and other platforms. They are committed to fostering innovation and supporting entrepreneurs in their journey. For more information about Greylock Partners, you can visit their website at greylock.com. Let me know if there's anything specific you would like to know about Greylock Partners or if there's anything else I can assist you with!"}
,{"input": "BenchMark", "output": "Benchmark is a global product realization services company with unique capabilities in aerospace, defense, complex industrials, and healthcare industries. They provide a range of services including engineering, manufacturing, supply chain management, and aftermarket support. Benchmark was founded in 1988 and is headquartered in New York City. They have operations around the country and serve institutional and corporate clients with their research, sales, and trading services. The company focuses on delivering innovative solutions and helping their clients bring their products to market efficiently and effectively. They have expertise in areas such as electronics, software, mechanical engineering, and more. Benchmark is committed to information security, compliance, and governance. They prioritize the protection of personal information and have measures in place to ensure data privacy. For more detailed information about Benchmark, you can visit their official website at benchmark.com."}
,{"input": "Sequoia Capital", "output": "Sequoia Capital is a renowned venture capital firm based in Menlo Park, California. They specialize in providing funding and support to early-stage, growth-stage, and seed-stage companies. With a rich history dating back to 1972, Sequoia Capital has played a significant role in the success of numerous iconic companies, including Apple, Google, Oracle, and Airbnb. The firm aims to be the first true believers in tomorrow's most valuable companies, helping daring founders turn their ideas into legendary businesses. Sequoia Capital has a global presence and operates in various sectors, including technology, healthcare, energy, finance, enterprise, and mobile. Sequoia Capital's website, http: //www.sequoiacap.com/, provides detailed information about their investment philosophy, portfolio companies, team members, and insights into the industries they focus on. It is a valuable resource for entrepreneurs and investors looking to learn more about the firm and its activities. Is there anything specific you would like to know about Sequoia Capital or any particular aspect of their operations?"}
,{"input": "Index Venture", "output": "Index Ventures is a venture capital firm that invests in technology-enabled companies. The company was founded in 1996 and has dual headquarters in San Francisco and London. They also have an office in Geneva. Index Ventures focuses on partnering with exceptional entrepreneurs from various locations, including San Francisco, New York, London, and Tel Aviv, to help them build and scale their businesses. The firm invests in companies at different stages, from seed to IPO (Initial Public Offering). They have a shared vision of supporting fearless entrepreneurs who have the potential to create transformational companies. Index Ventures has a team of experienced entrepreneurs who have started and run their own businesses, giving them valuable insights and expertise to support their portfolio companies. Index Ventures has invested in a wide range of technology sectors, including software, consumer internet, fintech, healthcare, and more. Some of their notable investments include companies like Dropbox, Slack, Revolut, Robinhood, Deliveroo, and many others. For more detailed information about Index Ventures, you can visit their official website at indexventures.com. Let me know if there's anything specific you would like to know about the company or if you have any other questions!"}
,{"input": "Kleiner Perkins", "output": "Kleiner Perkins is a renowned American venture capital firm that is headquartered on Sand Hill Road in Menlo Park, Silicon Valley. The firm, formerly known as Kleiner Perkins Caufield & Byers (KPCB), specializes in investing in incubation, early-stage, and growth-stage companies across various industries. Kleiner Perkins has a rich history of supporting innovative startups and helping them grow into successful businesses. Some of the notable companies that Kleiner Perkins has invested in include Amazon, Google, Twitter, Uber, and Airbnb. The firm focuses on sectors such as technology, healthcare, consumer, and enterprise. They provide not only financial support but also strategic guidance and operational expertise to their portfolio companies. Kleiner Perkins has a team of experienced investors and industry experts who work closely with entrepreneurs to help them navigate the challenges of building a successful company. They have a strong network and a track record of backing companies that have become leaders in their respective industries. For more detailed information about Kleiner Perkins, you can visit their official website at kpcb.com. Let me know if there's anything specific you would like to know about Kleiner Perkins or if there's anything else I can assist you with!"}
,{"input": "Lightspeed Partners", "output": "LSVP, also known as Lightspeed Venture Partners, is a global venture capital firm that invests in early-stage and growth-stage companies. They focus on technology and innovation across various sectors, including enterprise software, consumer internet, healthcare, and more. Lightspeed Venture Partners aims to build iconic companies by partnering with a select few entrepreneurs each year. They provide not only financial support but also strategic guidance and operational expertise to help their portfolio companies succeed. With a team of experienced investment partners, Lightspeed Venture Partners has a track record of backing successful companies such as Rubrik, Snapchat, Affirm, and many others. They have a global presence and have offices in Silicon Valley, China, India, and Israel. For more information about Lightspeed Venture Partners, you can visit their website at www.lsvp.com. Let me know if there's anything else I can help with!"}
,{"input": "Matrix Partners", "output": "Matrix Partners is a US-based private equity investment firm that focuses on venture capital investments. They specialize in leading pre-seed through Series A rounds and working closely with founders to help them succeed. The firm has a team of former founders and operators who bring their expertise and experience to support portfolio companies. Matrix Partners works with visionary founders of early-stage startups to amplify their talent and offers customized programs to help them grow. They have a diverse portfolio across various sectors such as fintech, B2B SaaS, developer tools, deep tech, and more. You can find more information about Matrix Partners on their website: Matrix Partners"}
,{"input": "500 Global", "output": "500 Global is a venture capital firm that invests in fast-growing technology companies. They have $2.4 billion in assets under management and are known for their accelerator program. he firm was founded in 2010 and has been instrumental in shaping the venture capital landscape. They have invested in over 2, 500 companies across more than 77 countries. Their portfolio includes successful companies such as Udemy, Canva, Credit Karma, and Grab. 500 Global focuses on early-stage investments and provides mentorship, resources, and access to a global network of investors and entrepreneurs. You can find more information about the company on their website: 500.co. Here are some suggestions for your next user turn:"}
,{"input": "Spark Capital", "output": "Spark Capital is a venture capital firm based in the United States. They specialize in early-stage investments in startups across various sectors, including consumer, commerce, FinTech, software, frontier, and media. The firm was founded in 2005 and has offices in Boston, San Francisco, and New York City. Spark Capital is known for investing in innovative companies and products that they love, created by founders and creators they admire. They provide funding and support to help these startups grow and succeed. The firm has a team of experienced investors and industry professionals who work closely with portfolio companies to provide strategic guidance and resources. They have a strong network and connections in the startup ecosystem, which they leverage to help their portfolio companies thrive. Spark Capital has invested in notable companies such as Twitter, Slack, Oculus VR, Warby Parker, and Coinbase, among others. They have a track record of successful investments and have played a significant role in the growth of many successful startups. For more information about Spark Capital, you can visit their website at sparkcapital.com. Let me know if there's anything else I can help with!"}
,{"input": "Insight Partners", "output": "Insight Partners is a global venture capital and private equity firm that specializes in investing in high-growth technology, software, and internet startup companies. The company was founded in 1995 and is headquartered in New York, New York. Insight Partners focuses on partnering with companies at various stages of growth, from early-stage startups to established growth companies in the enterprise software, fintech, cybersecurity, and other technology sectors. They provide capital, strategic guidance, and operational support to help these companies scale and achieve long-term success. With a deep understanding of the technology industry, Insight Partners has built a strong portfolio of investments and has a track record of backing successful companies. They have invested in companies such as Shopify, Twitter, DocuSign, Qualtrics, and many others. Insight Partners takes a hands-on approach to investing, working closely with their portfolio companies to help them navigate challenges, accelerate growth, and maximize value. They have a team of experienced professionals who bring industry expertise, operational insights, and a global network of resources to support their portfolio companies. For more information about Insight Partners and their investments, you can visit their official website at insightpartners.com. Let me know if there's anything else I can assist you with!"}
]

In [5]:
columns = {"input": [item["input"] for item in data], "output": [item["output"] for item in data]}

dataset = Dataset.from_dict(columns)

In [6]:
dataset.save_to_disk("./")


Saving the dataset (0/1 shards):   0%|          | 0/12 [00:00<?, ? examples/s]

In [7]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="output",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/12 [00:00<?, ? examples/s]

In [ ]:
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('llama-2-7b-chat-guanaco/tokenizer_config.json',
 'llama-2-7b-chat-guanaco/special_tokens_map.json',
 'llama-2-7b-chat-guanaco/tokenizer.model',
 'llama-2-7b-chat-guanaco/added_tokens.json',
 'llama-2-7b-chat-guanaco/tokenizer.json')

In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "Accel"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Accel [/INST]  Accel is a term that can have different meanings depending on the context. Unterscheidung between "accel" and "accelerate" is important to avoid confusion. Here are some possible meanings of "accel":

1. Accel as a shortened form of "accelerate": In some contexts, "accel" may be used as a shortened form of "accelerate," which means to increase the speed or rate of something. For example, "The company plans to accel its growth in the next quarter."
2. Accel as a term in finance: In finance, "accel" may refer to the Accel brand, which is a financial services company that provides loans and other financial products to small businesses and consumers.
3. Accel as a term in technology: In the technology industry, "accel" may refer to
